In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import cross_validation
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

c:\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:


In [2]:
ds = pd.read_csv("../data/Census01.csv")

In [3]:
ds.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,class
count,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000
mean,38.643585,189664.134597,10.078089,1079.067626,87.502314,40.422382,0.239282
std,13.710510,105604.025423,2.570973,7452.019058,403.004552,12.391444,0.426649
min,17.000000,12285.000000,1.000000,0.000000,0.000000,1.000000,0.000000
25%,28.000000,117550.500000,9.000000,0.000000,0.000000,40.000000,0.000000
50%,37.000000,178144.500000,10.000000,0.000000,0.000000,40.000000,0.000000
75%,48.000000,237642.000000,12.000000,0.000000,0.000000,45.000000,0.000000
max,90.000000,1490400.000000,16.000000,99999.000000,4356.000000,99.000000,1.000000


In [4]:
ds.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
class              int64
dtype: object

In [ ]:
ds["workclass"].unique()

In [ ]:
values = ds["workclass"].unique()
encoded_value = 0
for value in values:
    ds.loc[ds["workclass"] == value, "xxx_encoded_workclass"] = encoded_value
    encoded_value = encoded_value + 1

In [ ]:
ds.dtypes

In [ ]:
ds["xxx_encoded_workclass"].unique()

In [ ]:
print(ds.describe())

In [ ]:
ds.dtypes

In [ ]:
def get_encoded_variable_name(variable_name):
    encoded_variable_name = "___" + "encoded" + "_" + variable_name
    return encoded_variable_name

def encode_variable(variable_name):
    encoded_variable_name = get_encoded_variable_name(variable_name)
  
    # Uses the "ordinal" of the variable values as the encoded form for the nominal variable
    values = ds[variable_name].unique()
    encoded_value = 0
    for value in values:
        ds.loc[ds[variable_name] == value, encoded_variable_name] = encoded_value
        encoded_value += 1

In [ ]:
nominal_variables = ["workclass","education","marital-status","occupation","relationship","race","sex","native-country"]

In [ ]:
for nominal_variable in nominal_variables:
    encode_variable(nominal_variable)

In [ ]:
ds.describe()

In [ ]:
numerical_variables = ["age","fnlwgt", "education-num", "capital-gain", "capital-loss", "hours-per-week"]

In [ ]:
target_variable = "class"

In [ ]:
all_variables = numerical_variables + list(map(get_encoded_variable_name, nominal_variables))

In [ ]:
split = cross_validation.ShuffleSplit(ds.shape[0], n_iter=1, random_state=1, test_size=.33)

In [ ]:
for estimation, validation in split:
    print("Estimation: {}".format(estimation))
    print("Validation: {}".format(validation))

In [ ]:
algorithm = LinearRegression()

In [ ]:
for estimation_indices, validation_indices in split:
    print("=============================================================================")
    print("Estimation variable: {}".format(ds[all_variables].iloc[estimation_indices,:]))
    print("Estimation target: {}".format(ds[target_variable].iloc[estimation_indices]))
    print("=============================================================================")
    print("Validation variables: {}".format(ds[all_variables].iloc[validation_indices,:]))
    print("Validation target: {}".format(ds[target_variable].iloc[validation_indices]))
    print("=============================================================================")
    

In [ ]:
    for estimation_indices, validation_indices in split:

        #
        # Extracts the records based on the split indices
        estimation_records = ds[all_variables].iloc[estimation_indices,:]
        estimation_target = ds[target_variable].iloc[estimation_indices]

        #
        # Computes the model
        algorithm.fit(estimation_records, estimation_target)

        #
        # Estimates the model performance by applying it to the validation dataset
        validation_predictions = algorithm.predict(ds[all_variables].iloc[validation_indices,:])

        validation_predictions[validation_predictions > .5] = 1
        validation_predictions[validation_predictions <= .5] = 0
        acc = accuracy_score(ds[target_variable].iloc[validation_indices], validation_predictions)
        print("Accuracy : {}".format(acc))
